# 5. Write a program to implement the naïve Bayesian classifier for a sample training data set stored as a .CSV file. Compute the accuracy of the classifier, considering few test data sets.

## Read and Handle Data

In [29]:
import csv
import random
import math

random.seed(0)
# 1.Data Handling
# 1.1 Loading the Data from csv file of Pima indians diabetes dataset.
def loadcsv(filename):
    reader = csv.reader(open(filename,"r"))
    dataset =[]
    for row in reader:
        inlist=[]
        for i in range(len(row)):
            # converting the attributes from string to floating point numbers
            inlist.append(float(row[i]))
        dataset.append(inlist)
    return dataset

#### Splitting the Data set into Training Set

In [30]:
#The naive bayes model is comprised of a 
#summary of the data in the training dataset. 
#This summary is then used when making predictions.
#involves the mean and the standard deviation for each attribute, by class value
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy)) # random index
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

## Summarize Data

#### Separate Data By Class

In [31]:
#Function to categorize the dataset in terms of classes 
#The function assumes that the last attribute (-1) is the class value. 
#The function returns a map of class values to lists of data instances.
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if vector[-1] not in separated:
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated
 

#### Calculate the Mean

In [32]:
#The mean is the central middle or central tendency of the data, 
# and we will use it as the middle of our gaussian distribution 
# when calculating probabilities

#2.2 : Calculate Mean
def mean(numbers):
    return sum(numbers)/float(len(numbers))

#### Calculate the Standard Deviation

In [33]:
#The standard deviation describes the variation of spread of the data, 
#and we will use it to characterize the expected spread of each attribute
#in our Gaussian distribution when calculating probabilities.

#2.3 : Calculate Standard Deviation
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

#### Summarize Dataset

In [34]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for 
                 attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

#### Summarize Attributes By Class

In [35]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    print('Training Set after separation:\n',separated)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

## Make Prediction

#### Calculate Probaility Density Function

In [36]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent


#### Calculate Class Probabilities

In [37]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *=calculateProbability(x, mean, stdev)
    return probabilities

#### Prediction : look for the largest probability and return the associated class

In [38]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries,inputVector)
    print('\n Probabilities',probabilities)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

#### Make Predictions

In [39]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

#### Computing Accuracy

In [40]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

#### Main Function

In [41]:
def main():
    filename = 'Gender.csv'
    splitRatio = 0.67
    dataset = loadcsv(filename)
    
    #print("\n The Data Set :\n",dataset)
    print("\n The length of the Data Set : ",len(dataset))
    
    print("\n The Data Set Splitting into Training and Testing \n")
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    print('\n Number of Rows in Training Set:{0} rows'
          .format(len(trainingSet)))
    print('\n Number of Rows in Testing Set:{0} rows'
          .format(len(testSet)))
    
    print("\n Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i],"\n")
    
    print("\n Three Rows of Testing Set:\n")
    for i in range(0,3):
        print(testSet[i],"\n")
   
    # prepare model
    summaries = summarizeByClass(trainingSet)
    print("\n Model Summaries:\n",summaries)
   
    # test model
    predictions = getPredictions(summaries, testSet)
    print("\nPredictions:\n",predictions)
    
    accuracy = getAccuracy(testSet, predictions)
    print('\n Accuracy: {0}%'.format(accuracy))
main()



 The length of the Data Set :  8

 The Data Set Splitting into Training and Testing 


 Number of Rows in Training Set:5 rows

 Number of Rows in Testing Set:3 rows

 Five Rows of Training Set:

[5.42, 130.0, 7.0, 1.0] 

[5.75, 150.0, 9.0, 1.0] 

[5.92, 165.0, 10.0, 0.0] 

[6.0, 180.0, 12.0, 0.0] 

[5.0, 100.0, 6.0, 1.0] 


 Three Rows of Testing Set:

[5.92, 190.0, 11.0, 0.0] 

[5.58, 170.0, 12.0, 0.0] 

[5.5, 150.0, 8.0, 1.0] 

Training Set after separation:
 {1.0: [[5.42, 130.0, 7.0, 1.0], [5.75, 150.0, 9.0, 1.0], [5.0, 100.0, 6.0, 1.0]], 0.0: [[5.92, 165.0, 10.0, 0.0], [6.0, 180.0, 12.0, 0.0]]}

 Model Summaries:
 {1.0: [(5.390000000000001, 0.37589892258425006), (126.66666666666667, 25.16611478423583), (7.333333333333333, 1.5275252316519465)], 0.0: [(5.96, 0.05656854249492385), (172.5, 10.606601717798213), (11.0, 1.4142135623730951)]}

 Probabilities {1.0: 3.8434834608517055e-06, 0.0: 0.014940574036280761}

 Probabilities {1.0: 8.257462900666094e-06, 0.0: 9.00860085927302e-12}

 P